In [97]:
from cust_functions.explain_helper import *
import pandas as pd 


In [98]:
# ResGCN on AKI data
Features_AKI_ResGCN = create_avg_sd_df('./explanations/AKI_ResGCN.pkl', True, True)
Pathways_AKI_ResGCN = create_avg_sd_df('./explanations/AKI_ResGCN.pkl', False, True)
Features_Pert_AKI_ResGCN = create_avg_sd_df('./explanations/Pert_AKI_ResGCN.pkl', True, True)
Pathways_Pert_AKI_ResGCN = create_avg_sd_df('./explanations/Pert_AKI_ResGCN.pkl', False, True)

# ResGCN on Covid data 
Features_Covid_ResGCN = create_avg_sd_df('./explanations/Covid_ResGCN.pkl', True, True)
Pathways_Covid_ResGCN = create_avg_sd_df('./explanations/Covid_ResGCN.pkl', False, True)
Features_Pert_Covid_ResGCN = create_avg_sd_df('./explanations/Pert_Covid_ResGCN.pkl', True, True)
Pathways_Pert_Covid_ResGCN = create_avg_sd_df('./explanations/Pert_Covid_ResGCN.pkl', False, True)

# ResGAT on AKI data
Features_AKI_ResGAT = create_avg_sd_df('./explanations/AKI_ResGAT.pkl', True, True)
Pathways_AKI_ResGAT = create_avg_sd_df('./explanations/AKI_ResGAT.pkl', False, True)

# ResGAT on Covid data
Features_Covid_ResGAT = create_avg_sd_df('./explanations/Covid_ResGAT.pkl', True, True)
Pathways_Covid_ResGAT = create_avg_sd_df('./explanations/Covid_ResGAT.pkl', False, True)


In [99]:
def common_features_nodes(df1 : pd.DataFrame, df2 : pd.DataFrame, protein : bool, out_of_top: int):

    if protein:
        protein_or_pathway = 'Protein'
    else: 
        protein_or_pathway = 'Pathway'

    merged_df = pd.merge(df1.head(out_of_top), df2.head(out_of_top), on=protein_or_pathway, how='inner')

    return merged_df

In [100]:
def overlap_matrix(list_of_scores : list, list_of_names : list, protein: bool, out_of_top: int):
    Pairwise_matrix = pd.DataFrame(index=list_of_names, columns=list_of_names)

    for i, df1 in enumerate(list_of_scores):
        for j, df2 in enumerate(list_of_scores):
            Pairwise_matrix.iloc[i,j] = len(common_features_nodes(df1, df2, protein, out_of_top))

    return(Pairwise_matrix)

## 1. GNN versus GNN

In [101]:
##
# AKI data / proteins
##

AKI_features = [Features_AKI_ResGCN, Features_Pert_AKI_ResGCN, Features_AKI_ResGAT]
names = ['AKI_ResGCN', 'Pert_AKI_ResGCN', 'AKI_ResGAT']
overlap_matrix(AKI_features, names, True, 30)

,AKI_ResGCN,Pert_AKI_ResGCN,AKI_ResGAT
AKI_ResGCN,30,12,14
Pert_AKI_ResGCN,12,30,8
AKI_ResGAT,14,8,30


In [102]:
##
# AKI data / pathways
##

AKI_pathways = [Pathways_AKI_ResGCN, Pathways_Pert_AKI_ResGCN, Pathways_AKI_ResGAT]
names = ['AKI_ResGCN', 'Pert_AKI_ResGCN', 'AKI_ResGAT']
overlap_matrix(AKI_pathways, names, False, 30)

,AKI_ResGCN,Pert_AKI_ResGCN,AKI_ResGAT
AKI_ResGCN,30,6,24
Pert_AKI_ResGCN,6,30,5
AKI_ResGAT,24,5,30


In [106]:
##
# Covid data / proteins
##

Covid_features = [Features_Covid_ResGCN, Features_Pert_Covid_ResGCN, Features_Covid_ResGAT]
names = ['Covid_ResGCN', 'Pert_Covid_ResGCN', 'Covid_ResGAT']
overlap_matrix(Covid_features, names, True, 30)

,Covid_ResGCN,Pert_Covid_ResGCN,Covid_ResGAT
Covid_ResGCN,30,17,16
Pert_Covid_ResGCN,17,30,19
Covid_ResGAT,16,19,30


In [ ]:
##
# Covid data / pathways
##

Covid_pathways = [Pathways_Covid_ResGCN, Pathways_Pert_Covid_ResGCN, Pathways_Covid_ResGAT]
names = ['Covid_ResGCN', 'Covid_AKI_ResGCN', 'Covid_ResGAT']
overlap_matrix(AKI_pathways, names, False, 30)

## 2. GNN versus BINN